In [18]:
# 1. open tracklet file
# 2. group tracklets by entity
# 3. extract all tracklets images for a game
# 4. extract all raw results using florence

import os
import pandas as pd
import numpy as np
from glob import glob

tracklet_fps = glob("/mnt/sun/levlevi/nba-plus-statvu-dataset/player-tracklets" + "/*.txt")
ex = tracklet_fps[0]

def tracklet_to_df(fp: str):
    with open(ex, "r") as f:
        lines = f.readlines()
    data = [line.strip().split(",") for line in lines]
    data_array = np.array(data, dtype=float)[:, :6] # Convert to float for numerical operations
    # `frame, id, x1, y1, width, height, pad, pad, pad`
    df = pd.DataFrame(
        data_array,
        columns=[
            "frame",
            "id",
            "x1",
            "y1",
            "width",
            "height",
        ]
    )
    return df

In [20]:
df = tracklet_to_df(ex)
df.head()

,frame,id,x1,y1,width,height
0,61.0,1.0,485.6,268.6,73.4,107.0
1,61.0,2.0,966.2,255.7,73.5,127.8
2,61.0,3.0,889.3,228.3,65.4,110.7
3,62.0,1.0,489.3,265.6,65.3,108.3
4,62.0,2.0,962.0,254.0,72.1,133.1


In [22]:
def remove_short_tracklets(df, min_frames=30):
    id_counts = df.sort_values(by=['id', 'frame'])
    # Initialize a list to keep track of valid IDs
    valid_ids = []
    # Iterate over the grouped data by 'id'
    for id, group in id_counts.groupby('id'):
        # Check for consecutive frames
        group['frame_diff'] = group['frame'].diff().fillna(1)
        consecutive_frames = (group['frame_diff'] == 1).astype(int).groupby(group['frame_diff'].ne(1).cumsum()).cumsum()
        # Check if there are at least 30 consecutive frames
        if consecutive_frames.max() >= min_frames:
            valid_ids.append(id)
    # Filter the DataFrame to keep only valid IDs
    filtered_df = df[df['id'].isin(valid_ids)]
    filtered_df
    return df

In [ ]:
# 305290
# 303337

df_filtered = remove_short_tracklets(df)
